In [1]:
import os
import time
import datetime
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPool2D, Dropout,MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10

from sklearn.utils import check_random_state
#from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.datasets import cifar10

def get_dataset():
  (X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()
  # summarize loaded dataset
  print('Cifar Train: X=%s, y=%s' % (X_train_full.shape, y_train_full.shape))
  print('Cifar Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
  return (X_train_full, y_train_full, X_test, y_test)

class Normalize(object):
    
    def normalize(self, X, y):
        X = X.astype('float32')/255.
        y = y.astype('int32')
        return (X, y) 
    
    def inverse(self, X, y):
        X = X.astype('float32')*255.
        y = y.astype('int32')
        return (X, y) 

In [3]:
from sklearn.utils import check_random_state
import numpy as np

def get_k_random_samples(trainset_size, initial_labeled_samples, X_train_full,
                         y_train_full):
    random_state = check_random_state(0)
    permutation = np.random.choice(trainset_size, initial_labeled_samples, replace=False)

    print ('Initial random chosen samples', permutation.shape)
    X_train = X_train_full[permutation]
    y_train = y_train_full[permutation]
    #X_train = X_train.reshape((X_train.shape[0], -1))
    y_train_bin = y_train.reshape(initial_labeled_samples,)
    bin_count = np.bincount(y_train_bin.astype('int64'))
    unique = np.unique(y_train)
    print ( 'initial train set:', X_train.shape, y_train.shape,
        '\nlabels count:', bin_count, unique,
        )
    return (permutation, X_train, y_train)

In [4]:
import numpy as np
from scipy.stats import entropy
from sklearn.utils import check_random_state

labels= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class RandomSelection:
    def __init__(self):
      pass

    #@staticmethod
    def select(self, probas_val, initial_labeled_samples):
        random_state = check_random_state(0)
        selection = np.random.choice(probas_val.shape[0], initial_labeled_samples, replace=False)
        return selection

class QBC:

    def __init__(self):
        pass
    def average_KL_divergence(self, probas_val, probas_val2, X_train, y_train, X_seedset, y_seedset, samples):
      preds = []
      preds.append(probas_val)
      preds.append(probas_val2)

      #change normal arry to numpy array with stack
      consensus = np.mean(np.stack(preds), axis=0)
      print('consensus :',consensus)
      divergence = []
      for y_out in preds:
        divergence.append(entropy(consensus.T, y_out.T))

      result = np.apply_along_axis(np.mean, 0, np.stack(divergence))

      #argsort Returns the indices that would sort an array.
      #argsort sort in increasing order but if use (-result) then it sort in decreasing order.
      rankings = np.argsort(-result)[:samples]

      return rankings

    def vote_entropy(self, probas_val, probas_val2, X_train, y_train, X_seedset, y_seedset, samples):
      #vote entropy
      preds = []

      probas_val_not_cat = np.argmax(probas_val,axis=1)
      probas_val2_not_cat = np.argmax(probas_val2, axis=-1)

      preds.append(np.eye(len(labels))[probas_val_not_cat])
      preds.append(np.eye(len(labels))[probas_val2_not_cat])

      # C = no of models
      C = 2
      votes = np.apply_along_axis(np.sum, 0, np.stack(preds)) / C
      results = np.apply_along_axis(entropy, 1, votes)

      rankings = np.argsort(-results)[:samples]

      return rankings

class uncertainty_sampling:
    def init(self):
      pass

    def least_confident(self, probs, samples):
        #get the least uncertain value by subtracting the most uncertain value from 100% or 1
        scores = 1 - np.amax(probs, axis=1)
        #get the index of the least uncertain 
        rankings = np.argsort(-scores)[:samples]
        return rankings
    
    def max_margin(self, probs, samples):
        margin = np.partition(-probs, 1, axis=1)
        scores = -np.abs(margin[:,0] - margin[:, 1])
        rankings = np.argsort(-scores)[:samples]
        return rankings

    def entropy(self, probs, samples):
        socres = np.apply_along_axis(entropy,1, probs)
        rankings = np.argsort(-scores)[:samples]
        return rankings

In [5]:
import tensorflow as tf
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.callbacks import ModelCheckpoint
#%load_ext tensorboard

def train_model1( X_train_org, y_train_org,X_seedset, X_test, y_test, labels, iterations):
  
        print("\nModel 1 training")

        X_train, X_val, y_train, y_val = train_test_split( X_train_org, y_train_org, test_size=0.3, random_state=42)
        print('X_train :', X_train.shape)
        print('y train :', y_train.shape)
        print('X val :', X_val.shape)
        print('y val:', y_val.shape)

        # filepath = 'modelfiles/model_1.h5'
        # cp = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

        # log_dir = "logs/fit/model1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        model = Sequential()

        model.add(Conv2D(filters=32, kernel_size=(4, 4), input_shape=(32, 32, 3), activation='relu', padding='same'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(filters=64, kernel_size=(4, 4), activation='relu', padding='same'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(filters=128, kernel_size=(4, 4), activation='relu', padding='same'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(10, activation='softmax'))

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        early_stop = EarlyStopping(monitor='val_loss', patience=2)

        r = model.fit(X_train, y_train,  epochs=50, validation_data=(X_val, y_val), verbose=0)#, callbacks=[cp, tensorboard_callback])
        
        plt.figure(figsize=(12, 8))
        plt.subplot(2, 2, 1)
        plt.plot(r.history['loss'], label='Loss')
        plt.plot(r.history['val_loss'], label='val_Loss')
        plt.legend()
        plt.title('Loss evolution')
        plt.savefig('images/'+str(iterations)+'_model1_loss.png')

        plt.subplot(2, 2, 2)
        plt.plot(r.history['accuracy'], label='accuracy')
        plt.plot(r.history['val_accuracy'], label='val_accuracy')
        plt.legend()
        plt.title('Accuracy evolution')
        plt.savefig('images/'+str(iterations)+'_model1_acc.png')

        # Evaluate the model on the test data using `evaluate`
        print('\n# Evaluate on test data')
        results = model.evaluate(X_test, y_test, batch_size=128)
        print('test loss, test acc:', results)
        
        test_y_predicted = model.predict(X_test)
        test_y_predicted = np.argmax(test_y_predicted, axis=1)
        val_y_predicted  = model.predict(X_val)
        val_y_predicted = np.argmax(val_y_predicted, axis=1)
        cm = confusion_matrix(y_test, test_y_predicted)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        fig, ax = plt.subplots(figsize=(10, 10))
        disp = disp.plot(xticks_rotation='vertical', ax=ax,cmap='summer')
        plt.show()
        plt.savefig('images/'+str(iterations)+'_model1_cn.png')

        probas_val = model.predict(X_seedset)
        #print ('probabilities:', probas_val.shape, '\n', np.argmax(probas_val, axis=1))

        recordmodel1.loc[iterations] = [X_train_org.shape[0], X_seedset.shape[0], r.history['accuracy'][-1], r.history['loss'][-1],r.history['val_accuracy'][-1],r.history['val_loss'][1], results[1], results[0]]

        return probas_val

Using TensorFlow backend.


In [6]:
def train_model2( X_train_org, y_train_org,X_seedset, X_test, y_test, labels, iterations):

        print("\nModel 2 training")
        
        X_train, X_val, y_train, y_val = train_test_split( X_train_org, y_train_org, test_size=0.3, random_state=42)
        print('X_train :', X_train.shape)
        print('y train :', y_train.shape)
        print('X val :', X_val.shape)
        print('y val:', y_val.shape)

        # filepath = 'modelfiles/model_2.h5'
        # cp = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

        # log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        model = tf.keras.Sequential([
            
            Conv2D(48, kernel_size=(3,3), activation='relu', padding='same', input_shape=(32,32,3)),
            MaxPooling2D((2,2),(2,2)),
            
            Conv2D(96, kernel_size=(3,3), activation='relu', padding='same'),
            MaxPooling2D((2,2),(2,2)),
            
            Conv2D(192,kernel_size=(3,3), activation='relu', padding='same'),
            Conv2D(192,kernel_size=(3,3), activation='relu', padding='same'),
            MaxPooling2D((2,2),(2,2)),
            
            Conv2D(256,kernel_size=(3,3), activation='relu', padding='same'),
            MaxPooling2D((2,2),(2,2)),

            Flatten(),
            Dense(512, activation='tanh'),
            Dense(256, activation='tanh'),
            Dense(10, name='logits'),
            Activation('softmax')
        ])

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        early_stop = EarlyStopping(monitor='val_loss', patience=2)

        r = model.fit(X_train, y_train,  epochs=50, validation_data=(X_val, y_val), verbose=0)#, callbacks=[cp, tensorboard_callback])

        plt.figure(figsize=(12, 8))

        plt.subplot(2, 2, 1)
        plt.plot(r.history['loss'], label='Loss')
        plt.plot(r.history['val_loss'], label='val_Loss')
        plt.legend()
        plt.title('Loss evolution')
        plt.savefig('images/'+str(iterations)+'_model2_loss.png')

        plt.subplot(2, 2, 2)
        plt.plot(r.history['accuracy'], label='accuracy')
        plt.plot(r.history['val_accuracy'], label='val_accuracy')
        plt.legend()
        plt.title('Accuracy evolution')
        plt.savefig('images/'+str(iterations)+'_model2_acc.png')

        # Evaluate the model on the test data using `evaluate`
        print('\n# Evaluate on test data')
        results = model.evaluate(X_test, y_test, batch_size=128)
        print('test loss, test acc:', results)
        
        test_y_predicted = model.predict(X_test)
        test_y_predicted = np.argmax(test_y_predicted, axis=1)
        val_y_predicted  = model.predict(X_val)
        val_y_predicted = np.argmax(val_y_predicted, axis=1)
        cm = confusion_matrix(y_test, test_y_predicted)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        fig, ax = plt.subplots(figsize=(10, 10))
        disp = disp.plot(xticks_rotation='vertical', ax=ax,cmap='summer')
        plt.show()
        plt.savefig('images/'+str(iterations)+'model2_cn.png')

        probas_val = model.predict(X_seedset)
        #print ('probabilities:', probas_val.shape, '\n', np.argmax(probas_val, axis=1))

        recordmodel2.loc[iterations] = [X_train_org.shape[0], X_seedset.shape[0], r.history['accuracy'][-1], r.history['loss'][-1],r.history['val_accuracy'][-1],r.history['val_loss'][1], results[1], results[0]]

        return probas_val

In [7]:
def plot_fun(X, y):
  #X_train, X_test = X_train.astype('int32')*255. , X_test.astype('int8')*255.

  # Define the labels of the dataset
  labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
            'dog', 'frog', 'horse', 'ship', 'truck']

  # Let's view more images in a grid format
  # Define the dimensions of the plot grid 
  W_grid = 10
  L_grid = 10

  # fig, axes = plt.subplots(L_grid, W_grid)
  # subplot return the figure object and axes object
  # we can use the axes object to plot specific figures at various locations

  fig, axes = plt.subplots(L_grid, W_grid, figsize = (17,17))

  axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

  n_train = len(X) # get the length of the train dataset

  # Select a random number from 0 to n_train
  for i in np.arange(0, W_grid * L_grid): # create evenly spaces variables 

      # Select a random number
      index = np.random.randint(0, n_train)
      # read and display an image with the selected index    
      axes[i].imshow(X[index,1:])
      label_index = int(y[index])
      axes[i].set_title(labels[label_index], fontsize = 8)
      axes[i].axis('off')

  plt.subplots_adjust(hspace=0.4)

In [ ]:
labels= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

#define no of samples (to select from pool or seed set) per round
samples = 2000
trainset_size = 50000

#store the accuray and losses in a dataframe
recordmodel1 = pd.DataFrame(columns=('TrainDS','Seedset', 'Train_Accuracy', 'Train_Loss', 'Val_Accuracy', 'Val_Loss', 'Test_Accuracy', 'Test_Loss' ))
recordmodel2 = pd.DataFrame(columns=('TrainDS','Seedset', 'Train_Accuracy', 'Train_Loss', 'Val_Accuracy', 'Val_Loss', 'Test_Accuracy', 'Test_Loss' ))

#get cifar dataset
X_train_full, y_train_full, X_test, y_test = get_dataset()

#get initial 2000 samples
permutation, X_train, y_train = get_k_random_samples(X_train_full.shape[0],samples, X_train_full, y_train_full)

print("Train set size X :",X_train.shape)
print(y_train.shape)

#deine the seedset or pool
X_seedset = np.array([])
y_seedset = np.array([])
X_seedset = np.copy(X_train_full)
X_seedset = np.delete(X_seedset, permutation, axis=0)
y_seedset = np.copy(y_train_full)
y_seedset = np.delete(y_seedset, permutation, axis=0)
print ('Seed set (Pool) :', X_seedset.shape, y_seedset.shape)

#Normalize
normalizer = Normalize()
X_train, y_train = normalizer.normalize( X_train, y_train)
X_test, y_test = normalizer.normalize(X_test, y_test)
X_seedset, y_seedset = normalizer.normalize(X_seedset, y_seedset)

#train model
#if u r using QBC approach, use both models.
#if  r using uncertainty sampling, use only one model.
iterations = 0
probas_val1 = train_model1(X_train, y_train, X_seedset,X_test, y_test, labels, iterations)
probas_val2 = train_model2(X_train, y_train, X_seedset, X_test, y_test, labels, iterations)

#choose uncertain samples
#QBC
#qbc = QBC()
#selection_ranking = qbc.vote_entropy(probas_val1, probas_val2,X_train, y_train, X_seedset, y_seedset, samples)

#uncertainty sampling 
us = uncertainty_sampling()
#pass the predict values of the desired model as first parameter 
selection_ranking = us.least_confident(probas_val1, samples)

#random selection
#rs = RandomSelection()
#random_selection = rs.select(probas_val, samples)

#get how many uncertain samples are selected from which classes
selected_samples = y_seedset[selection_ranking]
selected_samples = selected_samples.reshape((selected_samples.shape[0],))
bin_count = np.bincount(selected_samples)
unique = np.unique(selected_samples)
print ('Selected Uncertainty samples :', bin_count, "from each classes ",unique )

#add the selected samples to the training set
X_train = np.concatenate((X_train, X_seedset[selection_ranking, :]))
y_train = np.concatenate((y_train, y_seedset[selection_ranking]))

#delete selected sampling from the seeed set or pool
X_seedset = np.delete(X_seedset, selection_ranking, axis=0)
y_seedset = np.delete(y_seedset, selection_ranking, axis=0)

print('\nAfter selecting samples based on Samping methods on round',iterations+1,' :')
print('Train : ', X_train.shape, y_train.shape)
print('Seedset  : ', X_seedset.shape, y_seedset.shape)

#print the total number of samples in each class in training set
y_train_bin = y_train.reshape((y_train.shape[0],))
bin_count = np.bincount(y_train_bin.astype('int64'))
unique = np.unique(y_train_bin.astype('int64'))
print ('Total number of samples :', bin_count, ' in each classes',unique )

iterations = 1
while len(X_seedset) > 1 :
    print('\n-------Round ',iterations+1,'----------------')
    #normalize
    #   normalizer = Normalize()
    #   X_train, y_train = normalizer.normalize( X_train, y_train)
    #   X_test, y_test = normalizer.normalize(X_test, y_test)    
    y_train_bin = y_train.reshape((y_train.shape[0],))
    bin_count = np.bincount(y_train_bin.astype('int64'))
    unique = np.unique(y_train_bin.astype('int64'))
    print ('Total number of samples :', bin_count, ' in each classes',unique )  
    #   X_seedset, y_seedset = normalizer.normalize(X_seedset, y_seedset)

    #train
    probas_val1 = train_model1(X_train, y_train, X_seedset,X_test, y_test, labels, iterations)
    probas_val2 = train_model2(X_train, y_train, X_seedset, X_test, y_test, labels, iterations)


    #get uncertain examples
    selection_ranking = us.least_confident(probas_val1, samples)

    # normalization needs to be inversed and recalculated based on the new train and test set.
    #   X_train, y_train = normalizer.inverse(X_train, y_train)
    #   X_test, y_test = normalizer.inverse(X_test, y_test)
    #   X_seedset, y_seedset = normalizer.inverse(X_seedset, y_seedset)
    X_train = np.concatenate((X_train, X_seedset[selection_ranking, :]))
    y_train = np.concatenate((y_train, y_seedset[selection_ranking]))

    X_seedset = np.delete(X_seedset, selection_ranking, axis=0)
    y_seedset = np.delete(y_seedset, selection_ranking, axis=0)

    print('After selecting samples based on Samping methods on round',iterations+1,' :')
    print('Train : ', X_train.shape, y_train.shape)
    print('Seedset  : ', X_seedset.shape, y_seedset.shape)

    y_train_bin = y_train.reshape((y_train.shape[0],))
    bin_count = np.bincount(y_train_bin.astype('int64'))
    unique = np.unique(y_train_bin.astype('int64'))
    print ('Total number of samples :', bin_count, ' in each classes',unique )
    print('-------Finish training anthor 1000 samples on round ',iterations+1,'----------------')
    iterations += 1
    

Cifar Train: X=(50000, 32, 32, 3), y=(50000, 1)
Cifar Test: X=(10000, 32, 32, 3), y=(10000, 1)
Initial random chosen samples (2000,)
initial train set: (2000, 32, 32, 3) (2000, 1) 
labels count: [182 211 216 211 201 197 182 211 197 192] [0 1 2 3 4 5 6 7 8 9]
Train set size X : (2000, 32, 32, 3)
(2000, 1)
Seed set (Pool) : (48000, 32, 32, 3) (48000, 1)

Model 1 training
X_train : (1400, 32, 32, 3)
y train : (1400, 1)
X val : (600, 32, 32, 3)
y val: (600, 1)


In [ ]:
recordmodel1